### Методика подготовки кастомизированного Allowed List на основе файла от Кастомера
С помощью данной методики можно:
- Подготовить команды для загрузки Combined Allowed листов для Кастомеров <br>
- Подготовить команды для загрузки STS листов для Кастомеров <br>
- Подготовить команды для пересоздания MCCMAP конфигурациии <br>

In [36]:
import numpy as np
import pandas as pd
from pandas import Series, DataFrame
import sys
from os.path import join, normpath
import os
import roamability as rb
import re
from collections import namedtuple
import datetime as dt

#######################################################################################
# Define variables here
#######################################################################################

# Input files

downloads = r'c:\Users\balob\Documents\GITLAB\RB_BD\DATA'
dmi_sts_list = 'Steering.sts_sponsor_country_export_Thu_Oct_29_2020.csv'
dmi_netpfx = 'DMI.dmi_netpfx_export_Wed_Jun_03_2020.csv' # используется для MCCMAP

max_data_rate_to_consider_discounted = 0.7

file_new_al = 'AL_STI.csv' # File name with the new Allowed list
# The example of the content of the file with the list of allowed networks:
# TADIG,SPONSOR
# ALBVF,S1
# ARGCM,S5
# ARM01,S2
# AUSOP,S1
# AUTHU,S2

# Output files
out_file_not_disc = 'AlNotDiscounted.csv'
out_file_toc_al_delete_not_disc = 'AlNotDiscountedDelete.rcjson'

# Задать название MCC MAP профайла
profile = 'STI_COMBINE_MAP'

# Задать имена для создаваемых, кастомизированных Allowed list
# Правило задания имен: <ИмяСпоносра>_TEST_COMB
# Эти Allowed Lists будут использоваться, также, для генерации MCC MAP профайла
s1_al_name = 'Partner_STI_Comb'
s2_al_name = 'P4_STI_Comb'
s4_al_name = 'MB_STI'
s5_al_name = 'SMART_STI_Comb'
s6_al_name = 'S6_STI_Comb'
s8_al_name = ''

# Задать имена для создаваемых, кастомизированных [sts] Sponsor Profile
# Правило задания имен: S1_TEST_STS
s1_sts_profile_name = 'Partner_STI'
s2_sts_profile_name = 'P4_STI'
s4_sts_profile_name = 'MB_STI'
s5_sts_profile_name = 'S5_STI_STS'
s6_sts_profile_name = 'S6_STI_STS'
s8_sts_profile_name = ''

# Задать имена для существующих "All" Allowed list (разрешены будут только те сети, которые присутствуют в 'All')
s1_al_all_name = 'Partner_All'
s2_al_all_name = 'P4_All'
s4_al_all_name = 'MB_ALL'
s5_al_all_name = 'SMART'
s6_al_all_name = 'S6_ALL'
s8_al_all_name = 'S8_ALL'

# Задать имена для существующих, основных "Combined" Allowed list
s1_al_combined_def_name = 'Partner_Combined'
s2_al_combined_def_name = 'P4_Combined'
s4_al_combined_def_name = 'MB_Combined'
s5_al_combined_def_name = 'SMART_Combined'
s6_al_combined_def_name = 'S6_Combined'
s8_al_combined_def_name = 'P8_Combined'

#######################################################################################

#######################################################################################

sponsor_object = namedtuple('Sponsor',
                'sponsor sponsor_id al_all_name al_pattern al_name_combined_custom al_name_combined_def sts_profile_name')

s1_sponsor = sponsor_object('S1', 1, s1_al_all_name, r'[Pp][Aa][rtner]?[\s\w-]*|^S1_[\s\w-]*',
                            s1_al_name, s1_al_combined_def_name, s1_sts_profile_name)
s2_sponsor = sponsor_object('S2', 2, s2_al_all_name, r'[Pp]4[\s\w-]*|^S2_[\s\w-]*',
                            s2_al_name, s2_al_combined_def_name, s2_sts_profile_name)
s4_sponsor = sponsor_object('S4', 4, s4_al_all_name, r'[Mm][Bb][\s\w-]*|^S4_[\s\w-]*',
                            s4_al_name, s4_al_combined_def_name, s4_sts_profile_name)
s5_sponsor = sponsor_object('S5', 5, s5_al_all_name, r'[Ss][Mm][Aa][Rr][Tt][\s\w-]*|^S5_[\s\w-]*',
                            s5_al_name, s5_al_combined_def_name, s5_sts_profile_name)
s6_sponsor = sponsor_object('S6', 6, s6_al_all_name, r'^S6_[\s\w-]*',
                            s6_al_name, s6_al_combined_def_name, s6_sts_profile_name)
s8_sponsor = sponsor_object('S8', 8, s8_al_all_name, r'^S8_[\s\w-]*',
                            s8_al_name, s8_al_combined_def_name, s8_sts_profile_name)

sponsors = [s1_sponsor, s2_sponsor, s4_sponsor, s5_sponsor, s6_sponsor, s8_sponsor]

def prepare_toc_to_create_al(df, downloads, result_file_name):
    """df[['SPONSOR','TADIG_AL','MOC','3G_ONLY']], where SPONSOR - S1, S2..."""
    df = df[['SPONSOR','TADIG_AL','MOC','3G_ONLY']].dropna().drop_duplicates()
    df.MOC = df.MOC.astype('int')
    df[['3G_ONLY']] = df[['3G_ONLY']].astype('int')
    ouf = open(join(downloads, result_file_name), 'w')
    num_val = df.count()[0] - 1
    t1 = '{"params":{"sponsor":"'
    t2 = '","plmnCode":"'
    t3 = '","mo":"'
    t4 = '","mt":"1","sms":"1","data":"'
    t5 = '","moSec":"1","dataKb":"1"},"type":"create","caption":"dmi_allowed_list","objectId":"DMI.dmi_allowed_list","serviceId":"DMI"}'
    ouf.write('[')
    for i, [sponsor_name, tadig, moc, only_3g] in enumerate(df.values):
        for [sponsor, al_name_combined] in [[sponsor.sponsor, sponsor.al_name_combined_custom] for sponsor in sponsors]:
            if sponsor_name == sponsor:
                ouf.write(t1 + al_name_combined + t2 + tadig + t3 + str(moc) + t4 + str(only_3g) + t5)
                if i < num_val:
                    ouf.write(',')
    ouf.write(']')
    ouf.close()

def prepare_toc_to_delete_al(df, downloads, result_file_name):
    """df[['Sponsor','TADIG']], where Sponsor - Allowed list name like Partner_Combined, P4_Combined..."""
    ouf = open(join(downloads, result_file_name), 'w')
    num_val = df.count()[0] - 1
    t1='{"params":{"sponsor":"'
    t2='","plmnCode":"'
    t3='"},"type":"remove","caption":"dmi_allowed_list","objectId":"DMI.dmi_allowed_list","serviceId":"DMI"}'
    ouf.write('[')
    for i, [sponsor_name, tadig] in enumerate(df.values):
        ouf.write(t1 + sponsor_name + t2 + tadig + t3)
        if i < num_val:
            ouf.write(',')
    ouf.write(']')
    ouf.close()

def prepare_toc_to_create_sts(df, downloads, result_file_name):
    """df[['SPONSOR','MCC','COUNTRY_NAME']], where SPONSOR - S1, S2..."""
    ouf=open(join(downloads, result_file_name), 'w')
    num_val = df.count()[0]-1
    t1 = '{"params":{"sponsorProfile":"'
    t2 = '","countryName":"'
    t3 = '","mcc":"'
    t4 = '"},"type":"create","caption":"sts_sponsor_country","objectId":"Steering.sts_sponsor_country","serviceId":"Steering"}'
    ouf.write('[')
    for i, [sponsor_name, mcc, country] in enumerate(df.values):
        for [sponsor, sts_sponsor_name] in [[sponsor.sponsor, sponsor.sts_profile_name] for sponsor in sponsors]:
            if sponsor_name == sponsor:
                ouf.write(t1 + sts_sponsor_name + t2 + country + t3 + str(mcc) + t4)
                if i < num_val:
                    ouf.write(',')
    ouf.write(']')
    ouf.close()

def prepare_toc_to_delete_sts(df, downloads, result_file_name):
    """df[['SPONSOR','MCC','COUNTRY_NAME']], where SPONSOR - S1, S2..."""
    ouf=open(join(downloads, result_file_name), 'w')
    num_val = df.count()[0] - 1
    t1='{"params":{"sponsorProfile":"'
#     t2='","countryName":"'
    t3='","mcc":"'
    t4='"},"type":"remove","caption":"sts_sponsor_country","objectId":"Steering.sts_sponsor_country","serviceId":"Steering"}'
    ouf.write('[')
    for i, [sponsor_name, mcc, country] in enumerate(df.values):
        for [sponsor, sts_sponsor_name] in [[sponsor.sponsor, sponsor.sts_profile_name] for sponsor in sponsors]:
            if sponsor_name == sponsor:
#                 ouf.write(t1 + sts_sponsor_name + t2 + country + t3 + str(mcc) + t4)
                ouf.write(t1 + sts_sponsor_name + t3 + str(mcc) + t4)
                if i < num_val:
                    ouf.write(',')
    ouf.write(']')
    ouf.close()

def check_al_for_differences(df_al_current, df_al_new):
    """df_al_current[['Sponsor','TADIG', 'SPONSOR']], df_al_new[['Sponsor','TADIG', 'SPONSOR','MOC','3G_ONLY']]"""
    df = pd.merge(df_al_current[['Sponsor','TADIG','SPONSOR']],
                    df_al_new[['Sponsor','TADIG', 'SPONSOR','MOC','3G_ONLY']], how='outer',
                    on='TADIG', suffixes=['_CURRENT', '_NEW'])
    return df[(df.SPONSOR_CURRENT != df.SPONSOR_NEW)]

def get_df_al():
    """
    Get Allowed lists from DMO Oracle DB
    """
    sql_str = """
SELECT
s.name AS Sponsor, o.plmn_code AS TADIG, al.mo_price AS MOC, al.mb_price
FROM allowed_list_p al, sponsor_p s, oper_p o
WHERE al.sponsor_ref = s.ri
AND al.operator_ref = o.ri
"""
    with rb.OracleConnect('DMI', 'dd607605ce341', 'DMI') as cnxn:
        df_al = pd.read_sql_query(sql_str, cnxn, coerce_float=False)
    df_al.rename({'SPONSOR':'Sponsor', 'MB_PRICE':'3G_ONLY'}, axis=1, inplace=True)
    return df_al

In [37]:
# Получение данных TADIG
# Получение прайсов с флагом is_discounted = 1

sql_srt=\
'''
SELECT DISTINCT c.COUNTRY_NAME,n.NETWORK_NAME,t.TADIG_CODE AS TADIG
,mcc.MCC
,t.NETWORK_ID FROM RDB_TADIG_CODES t
LEFT JOIN RDB_NETWORKS n ON t.network_id = n.network_id
LEFT JOIN RDB_COUNTRIES c ON n.country_id = c.country_id
LEFT JOIN RDB_NETWORK_IMSI_PREFIXES mcc ON mcc.network_id = n.network_id
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_tadig = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)

###################################################
# Adjustments
df_tadig.drop(
    df_tadig[(df_tadig.TADIG=='YUGPM') & (df_tadig.NETWORK_ID==623)].index, inplace=True)
# c:\W_DATA_ROAM\ДОКУМЕНТАЦИЯ\PRICES\Partner_TURTK_YUGPM_not_discounted.msg
###################################################
    
sql_srt=\
'''
SELECT r.NETWORK_ID
,r.SPONSOR_ID AS SPONSOR
,r.data_rate
FROM ROAMING_PLAN_RULES r
WHERE r.roaming_plan_id IN (267,268,329,343,368,372)
--AND ((r.start_date < GETDATE() AND r.end_date is null) OR (r.start_date < GETDATE() AND r.end_date > GETDATE()))
AND r.end_date IS NULL
AND r.is_discounted = 1
'''

with rb.MssqlConnect('172.18.11.82', '10028', 'BSS', 'iKQVm40AZAmyRaw72LeY') as cnxn:
    df_disc = pd.read_sql_query(sql_srt, cnxn, coerce_float=False)
    
print('TADIGs and MCC with NETWORK_ID:')
display(df_tadig.head(3))

replace_sponsor_id_dict = dict((sponsor.sponsor_id, sponsor.sponsor) for sponsor in sponsors)
df_disc.SPONSOR = df_disc.SPONSOR.replace(replace_sponsor_id_dict)

print('\nHigh cost in Discounted Networks:')
print('!!!WILL BE REMOVED FROM ALLOWED LIST!!!')
display(df_disc[df_disc.data_rate > max_data_rate_to_consider_discounted])

# Remove high data priced networks
df_disc = df_disc[df_disc.data_rate <= max_data_rate_to_consider_discounted]

TADIGs and MCC with NETWORK_ID:


,COUNTRY_NAME,NETWORK_NAME,TADIG,MCC,NETWORK_ID
0,Afghanistan,Afghan Wireless Communication Company,AFGAW,412,4
1,Afghanistan,Areeba/MTN,AFGAR,412,5
2,Afghanistan,Etisalat,AFG55,412,6



High cost in Discounted Networks:
!!!WILL BE REMOVED FROM ALLOWED LIST!!!


,NETWORK_ID,SPONSOR,data_rate


In [38]:
# Create df for existing "All" allowed lists 
# Create df for new "Allowed list"

# Like expression for Allowed List

df_al = get_df_al()

replace_dict = dict((sponsor.al_pattern, sponsor.sponsor) for sponsor in sponsors)

df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace=replace_dict, regex=True)

df_al = pd.merge(df_al, df_tadig, how='left', on='TADIG')

# Check not found Allowed Lists

print('\nNot found Allowed Lists:\n')
for sponsor in df_al.SPONSOR.unique().tolist():
    if sponsor not in [sponsor.sponsor for sponsor in sponsors]:
        print(sponsor)
print('-' * 80)

sponsor_cols = ['Sponsor', 'SPONSOR','NETWORK_ID','TADIG','MOC','3G_ONLY']

df_al_s1 = df_al.loc[df_al.Sponsor.isin([s1_sponsor.al_all_name]), sponsor_cols]
df_al_s2 = df_al.loc[df_al.Sponsor.isin([s2_sponsor.al_all_name]), sponsor_cols]
df_al_s4 = df_al.loc[df_al.Sponsor.isin([s4_sponsor.al_all_name]), sponsor_cols]
df_al_s5 = df_al.loc[df_al.Sponsor.isin([s5_sponsor.al_all_name]), sponsor_cols]
df_al_s6 = df_al.loc[df_al.Sponsor.isin([s6_sponsor.al_all_name]), sponsor_cols]
df_al_s8 = df_al.loc[df_al.Sponsor.isin([s8_sponsor.al_all_name]), sponsor_cols]

df_al_s_list = [df_al_s1, df_al_s2, df_al_s4, df_al_s5, df_al_s6, df_al_s8]

print('\nAll Allowed lists (heads):')
for df_temp in df_al_s_list:
    display(df_temp.head(3))

df_new_al = pd.read_csv(join(downloads, file_new_al), sep=',')
df_new_al = pd.merge(df_new_al, df_tadig, how='left', on='TADIG')
df_new_al.drop_duplicates(inplace=True)
print('\nThe new allowed list uploaded from file enriched by MCC and NETWORK_ID:')
df_new_al.head(3)


Not found Allowed Lists:

--------------------------------------------------------------------------------

All Allowed lists (heads):


,Sponsor,SPONSOR,NETWORK_ID,TADIG,MOC,3G_ONLY
855,Partner_All,S1,630,MOZVC,1,0
861,Partner_All,S1,938,TJKIT,1,0
1234,Partner_All,S1,629,MOZVT,1,0


,Sponsor,SPONSOR,NETWORK_ID,TADIG,MOC,3G_ONLY
696,P4_All,S2,170,CPVCV,1,0
706,P4_All,S2,103,BIHER,1,0
1066,P4_All,S2,238,CZECM,1,0


,Sponsor,SPONSOR,NETWORK_ID,TADIG,MOC,3G_ONLY
1419,MB_ALL,S4,802,SAUVG,1,0
3246,MB_ALL,S4,49,AUSVF,1,0
3248,MB_ALL,S4,199,CHNCU,1,0


,Sponsor,SPONSOR,NETWORK_ID,TADIG,MOC,3G_ONLY
2492,SMART,S5,631,MMRPT,0,0
2493,SMART,S5,385,HKGPP,1,0
2494,SMART,S5,997,ARETC,0,0


,Sponsor,SPONSOR,NETWORK_ID,TADIG,MOC,3G_ONLY
1514,S6_ALL,S6,285,FIN2G,1,0
1519,S6_ALL,S6,147,CMR02,1,0
1990,S6_ALL,S6,707,PSEWM,1,0


,Sponsor,SPONSOR,NETWORK_ID,TADIG,MOC,3G_ONLY
1435,S8_ALL,S8,770,RUS01,1,0
1436,S8_ALL,S8,765,RUSBD,1,0
5533,S8_ALL,S8,281,FJIDP,1,0



The new allowed list uploaded from file enriched by MCC and NETWORK_ID:


,TADIG,SPONSOR,COUNTRY_NAME,NETWORK_NAME,MCC,NETWORK_ID
0,AFGEA,S5,Afghanistan,Etisalat,412,6
1,AFG55,S5,Afghanistan,Etisalat,412,6
2,ALBVF,S1,Albania,Vodafone,276,12


In [40]:
# Find Not Discounted in any Allowed Lists in DMI

df_temp = pd.merge(df_al, df_disc, how='left', on=['NETWORK_ID','SPONSOR'])
df_temp = df_temp[df_temp.data_rate.isnull()]
df_temp.to_csv(join(downloads, out_file_not_disc), index=False)

# Prepare commands to adjust the existing AL of customer by deleting the extras
prepare_toc_to_delete_al(df_temp[['Sponsor','TADIG']].dropna().drop_duplicates(), downloads, out_file_toc_al_delete_not_disc)

df_temp

,Sponsor,TADIG,MOC,3G_ONLY,SPONSOR,COUNTRY_NAME,NETWORK_NAME,MCC,NETWORK_ID,data_rate
6982,S6_ALL,CRITC,0,0,S6,Costa Rica,Telefonica de Costa Rica TC S A,712,222,NaN
6983,Partner_TCOM_COM,BGRMT,1,0,S1,NaN,NaN,NaN,NaN,NaN
8391,S6_ONLY_STI,CRITC,0,0,S6,Costa Rica,Telefonica de Costa Rica TC S A,712,222,NaN


In [28]:
# Save old Allowed List of Customer to file

now = dt.datetime.now()
sponsor_name_customized_list = [sponsor.al_name_combined_custom for sponsor in sponsors]
df_old_al = df_al.loc[df_al.Sponsor.isin(sponsor_name_customized_list), ['TADIG', 'SPONSOR']]
df_old_al.to_csv(join(downloads, f"{file_new_al[:-4]}_backup_{now.strftime('%y%m%d')}.csv"), index=False)
df_old_al.head()

,TADIG,SPONSOR
175,EGYK9,S4
186,CHNCU,S1
198,KORKF,S1
205,NPLM2,S1
217,NZLBS,S1


In [29]:
# Check mismatches with 'ALL' Allowed Lists in DMI and the new Customized Combined Allowed List
# Prepare a file with TADIGs for creating Allowed List

def check_allowed(sponsor, df_new, df_allowed):
    df = pd.merge(df_new[df_new.SPONSOR == sponsor], df_allowed, how='left', on=['NETWORK_ID','SPONSOR'],
                      suffixes=['', '_AL'])
    return df

df_al_new_create = DataFrame()

print('\nNot found in "*_All" allowed lists TAGIGs')
print('!!!WILL BE REMOVED FROM NEW ALLOWED LIST!!!\n')

for sponsor, df_al_s in zip(sponsors, df_al_s_list):
    print(f'Missing in {sponsor.al_all_name} Allowed list in DMI:')
    df_temp_al = check_allowed(sponsor.sponsor, df_new_al, df_al_s)
    df_temp_missing = df_temp_al[df_temp_al.Sponsor.isnull()]
    display(df_temp_missing)
    df_temp_missing.to_csv(join(downloads, f'{sponsor.sponsor.lower()}_al_missing.csv'),index=False)
    
    df_temp_al.dropna(subset=['TADIG_AL'], inplace=True)
    df_temp_al.MOC = df_temp_al.MOC.astype('int64')
    df_temp_al['3G_ONLY'] = df_temp_al['3G_ONLY'].astype('int64')
    
    df_al_new_create = df_al_new_create.append(df_temp_al, ignore_index=True, sort=False)

print(f'Create new Allowed list in DMI:')
df_al_new_create.to_csv(join(downloads, f'df_al_create.csv'), index=False)
df_al_new_create.head(3)


Not found in "*_All" allowed lists TAGIGs
!!!WILL BE REMOVED FROM NEW ALLOWED LIST!!!

Missing in Partner_All Allowed list in DMI:


,TADIG,SPONSOR,COUNTRY_NAME,NETWORK_NAME,MCC,NETWORK_ID,Sponsor,TADIG_AL,MOC,3G_ONLY
33,ETH01,S1,Ethiopia,Ethio Telecom,636,276,NaN,NaN,NaN,NaN
144,INDIB,S1,India,Idea Cellular-IB,405,419,NaN,NaN,NaN,NaN
152,INDSK,S1,India,Idea Cellular-SK,404,1211,NaN,NaN,NaN,NaN
155,HTICL,S1,Jamaica,Digicel Jamaica Limited,338,496,NaN,NaN,NaN,NaN
156,JAMDC,S1,Jamaica,Digicel Jamaica Limited,338,496,NaN,NaN,NaN,NaN
171,MCOK8,S1,Monaco,MonacoTel MC,212,1224,NaN,NaN,NaN,NaN
220,TURTC,S1,Turkey,Turkcell,286,971,NaN,NaN,NaN,NaN
221,TURTK,S1,Turkey,Turkcell,286,971,NaN,NaN,NaN,NaN


Missing in P4_All Allowed list in DMI:


,TADIG,SPONSOR,COUNTRY_NAME,NETWORK_NAME,MCC,NETWORK_ID,Sponsor,TADIG_AL,MOC,3G_ONLY
4,REU02,S2,Reunion,Orange,647,754,NaN,NaN,NaN,NaN


Missing in MB_ALL Allowed list in DMI:


...

,TADIG,SPONSOR,COUNTRY_NAME,NETWORK_NAME,MCC,NETWORK_ID,Sponsor,TADIG_AL,MOC,3G_ONLY
10,BWAVC,S6,Botswana,Orange Botswana Pty Ltd,652,107,NaN,NaN,NaN,NaN
18,CAF03,S6,Central African Rep,Orange Centrafrique,623,177,NaN,NaN,NaN,NaN
47,GLP01,S6,French Guiana,Orange Caribe,340,310,NaN,NaN,NaN,NaN
58,GINGS,S6,Guinea,Orange/Sonatel/Spacetel,611,371,NaN,NaN,NaN,NaN
66,ISLTL,S6,Iceland,Fjarskipti hf,274,406,NaN,NaN,NaN,NaN
78,KWTNM,S6,Kuwait,National Mobile Telecommunications Company K S C,419,521,NaN,NaN,NaN,NaN
82,LBNFL,S6,Lebanon,Mobile Interim Company 1 MIC1,415,542,NaN,NaN,NaN,NaN
83,LBNLC,S6,Lebanon,Mobile Interim Company 2 MIC2,415,541,NaN,NaN,NaN,NaN
97,NEROR,S6,Niger,Orange/Sahelc.,614,675,NaN,NaN,NaN,NaN
100,OMNNT,S6,Oman,Omani Qatari Telecommunication Company SAOG,422,696,NaN,NaN,NaN,NaN


Missing in S8_ALL Allowed list in DMI:


,TADIG,COUNTRY_NAME,NETWORK_NAME,MCC,Sponsor,SPONSOR,NETWORK_ID,TADIG_AL,MOC,3G_ONLY


Create new Allowed list in DMI:


,TADIG,SPONSOR,COUNTRY_NAME,NETWORK_NAME,MCC,NETWORK_ID,Sponsor,TADIG_AL,MOC,3G_ONLY
0,ALBVF,S1,Albania,Vodafone,276,12,Partner_All,ALBVF,1,0
1,AIACW,S1,Anguilla,Cable and Wireless,365,20,Partner_All,AIACW,1,0
2,ATGCW,S1,Antigua and Barbuda,C & W,344,23,Partner_All,ATGCW,1,0


In [30]:
# Check for different Sponsors in the same country

s1_temp = df_new_al.groupby('COUNTRY_NAME')['SPONSOR'].nunique()
s1_temp[s1_temp.values > 1]

Series([], Name: SPONSOR, dtype: int64)

In [32]:
# Check for not discounted in the new Allowed list

print('\nNot discounted TAGIGs in the new Allowed list')
print('!!!WILL BE REMOVED FROM THE NEW ALLOWED LIST!!!')

df_temp = pd.merge(df_new_al, df_disc, how='left', on=['NETWORK_ID', 'SPONSOR'])
df_temp[df_temp.data_rate.isnull()]


Not discounted TAGIGs in the new Allowed list
!!!WILL BE REMOVED FROM THE NEW ALLOWED LIST!!!


,TADIG,SPONSOR,COUNTRY_NAME,NETWORK_NAME,MCC,NETWORK_ID,data_rate
28,BTNTC,S5,Bhutan,TashiCell,402,98,NaN
64,ANTCT,S5,Curacao,Polycom N.V./ Digicel,362,229,NaN
97,GMBAC,S5,Gambia,Africell Gambia Limited,607,319,NaN
118,GUYUM,S5,Guyana,U Mobile Cellular Inc,738,377,NaN
192,INDSK,S1,India,Idea Cellular-SK,404,1211,NaN
282,REU02,S2,Reunion,Orange,647,754,NaN
299,SLEAC,S5,Sierra Leone,Lintel SL Ltd,619,813,NaN
317,SURDC,S5,Suriname,Digicel Suriname N V,746,882,NaN
333,TTODL,S5,Trinidad and Tobago,Digicel,374,965,NaN
337,TURTC,S1,Turkey,Turkcell,286,971,NaN


### Prepare the TOC commands to create the full new Allowed list for Customer

In [7]:
# Create file with TOC commands to create the full new Allowed list for Customer

result_file_name_al_create = 'al_create.rcjson' # file name with TOC comands for cerating the new Allowed list
df_temp = df_al_new_create[['SPONSOR','TADIG_AL','MOC','3G_ONLY']].dropna().drop_duplicates()
prepare_toc_to_create_al(df_temp, downloads, result_file_name_al_create)

### Check for the inconsystency with the Customer's Combined

In [9]:
# Проверить различия с существующим Allowed list Кастомера.
# Проверяются только те операторы, которые были найдены в "All" Allowed list спонсоров.

df_al_custom_current = df_al.loc[df_al.Sponsor.isin([sponsor.al_name_combined_custom for sponsor in sponsors]),
                           ['Sponsor', 'TADIG', 'SPONSOR']]

df_al_custom_new = df_al_new_create[['Sponsor','SPONSOR','TADIG_AL','MOC','3G_ONLY']].rename({'TADIG_AL':'TADIG'}, axis=1).dropna().drop_duplicates()

df_make_changes_custom_current = check_al_for_differences(df_al_custom_current, df_al_custom_new)

df_make_changes_custom_current[['TADIG','SPONSOR_CURRENT','SPONSOR_NEW']].sort_values(by='TADIG') \
# .to_csv(join(downloads, 'al_changes.csv'), index=False)

,TADIG,SPONSOR_CURRENT,SPONSOR_NEW
179,AUTCA,S2,S6
68,AUTON,S2,S6
86,AUTTR,S2,S6
36,AZEAF,S1,NaN
37,AZEBC,S1,NaN
...,...,...,...
93,URYTM,S1,NaN
26,USACG,S2,S6
249,USAW6,NaN,S6
218,VENMS,NaN,S1


### Prepare the TOC commands to adjust Allowed list for Customer

In [10]:
# Prepare commands to adjust the existing AL of customer by deleting the extras

delete_file_name = 'al_adjust_delete.rcjson' # file name with TOC comands for deleting the extras

df_temp = df_make_changes_custom_current[['Sponsor_CURRENT','TADIG']].dropna().drop_duplicates()

prepare_toc_to_delete_al(df_temp, downloads, delete_file_name)

In [11]:
# Prepare commands to adjust the existing AL of customer by createing missing operators in Allowed list

create_file_name = 'al_adjust_create.rcjson' # file name with TOC comands for creating missing operators in Allowed list

df_temp = df_make_changes_custom_current[['SPONSOR_NEW','TADIG','MOC','3G_ONLY']].rename({
    'SPONSOR_NEW':'SPONSOR', 'TADIG':'TADIG_AL'}, axis=1).dropna().drop_duplicates()

prepare_toc_to_create_al(df_temp, downloads, create_file_name)

### Prepare the TOC commands to create full new STS list for Customer

In [12]:
# Создать новые правила STS с нуля

result_file_name_sts_create = 'sts_create.rcjson'  # TOC comands for cerating STS list

df_temp = df_al_new_create[['SPONSOR','MCC','COUNTRY_NAME']].dropna().drop_duplicates()

prepare_toc_to_create_sts(df_temp, downloads, result_file_name_sts_create)

### Prepare the TOC commands to adjust STS list for Customer

In [12]:
# Проверить различия с существующим STS list Кастомера.

df_sts = pd.read_csv(join(downloads, dmi_sts_list))
df_sts.drop('Country name', axis=1, inplace=True)
df_sts.rename({'Sponsor profile':'STS_PROFILE_NAME'}, axis=1, inplace=True)

df_sts['SPONSOR'] = df_sts['STS_PROFILE_NAME'].replace(to_replace=replace_dict, regex=True)

df_sts_new = df_al_new_create[['SPONSOR','MCC']].dropna().drop_duplicates()
df_sts_new.MCC = df_sts_new.MCC.astype('int64')

df_sts_customer = df_sts.loc[df_sts.STS_PROFILE_NAME.isin([sponsor.sts_profile_name for sponsor in sponsors])]

df_sts_customer = pd.merge(df_sts_customer,
                 df_sts_new, how='outer', on='MCC',
                suffixes=['_CUSTOM_CURRENT', '_CUSTOM_NEW'])

df_mcc = df_tadig.loc[:,['COUNTRY_NAME','MCC']].dropna().drop_duplicates().groupby('MCC', as_index=False).first()
df_mcc.MCC = df_mcc.MCC.astype('int64')

df_sts_customer = pd.merge(df_sts_customer, df_mcc, how='left', on='MCC')

df_make_changes = df_sts_customer[(df_sts_customer.SPONSOR_CUSTOM_CURRENT != df_sts_customer.SPONSOR_CUSTOM_NEW)]
df_make_changes

,STS_PROFILE_NAME,MCC,SPONSOR_CUSTOM_CURRENT,SPONSOR_CUSTOM_NEW,COUNTRY_NAME
3,P4_STI,216,S2,S6,Hungary
21,Partner_STI,284,S1,S6,Bulgaria
22,Partner_STI,302,S1,S6,Canada
27,Partner_STI,712,S1,S6,Costa Rica
29,Partner_STI,370,S1,S6,Dominican Republic
...,...,...,...,...,...
136,NaN,612,NaN,S6,Ivory Coast
137,NaN,610,NaN,S6,Mali
138,NaN,212,NaN,S6,Monaco
139,NaN,744,NaN,S6,Paraguay


In [13]:
# Prepare commands to adjust the existing STS list by deleting the extras

delete_file_name = 'sts_adjust_delete.rcjson'

df_temp = df_make_changes[['SPONSOR_CUSTOM_CURRENT','MCC','COUNTRY_NAME']].rename({'SPONSOR_CUSTOM_CURRENT':'SPONSOR'}
                                                                                  , axis=1).dropna().drop_duplicates()
prepare_toc_to_delete_sts(df_temp, downloads, delete_file_name)

In [14]:
# Prepare commands to create missing countries in STS list

create_file_name = 'sts_adjust_create.rcjson'

df_temp = df_make_changes[['SPONSOR_CUSTOM_NEW','MCC','COUNTRY_NAME']].rename({'SPONSOR_CUSTOM_NEW':'SPONSOR'}, axis=1).dropna().drop_duplicates()

prepare_toc_to_create_sts(df_temp, downloads, create_file_name)

### После обновления AL обновить MCC-MAP файл

In [33]:
# Prechecks and preparations

df_al = get_df_al()

replace_dict = dict((sponsor.al_pattern, sponsor.sts_profile_name) for sponsor in sponsors)

df_al['SPONSOR'] = df_al['Sponsor'].replace(to_replace=replace_dict, regex=True)

df_al = df_al.loc[df_al.Sponsor.isin([sponsor.al_name_combined_custom for sponsor in sponsors]), ['SPONSOR','TADIG']]

# Prepare mccmnc list from netpfx

df_np=pd.read_csv(join(downloads, dmi_netpfx), usecols=['Operator','MCC'], dtype={'MCC':'str'}).drop_duplicates()
df_np.rename({'Operator':'TADIG'}, axis=1, inplace=True)

df_mcc_map = pd.merge(df_al, df_np, how='left', on='TADIG')

# Missing MCC

print('Missing MCC:')
display(df_mcc_map[df_mcc_map.MCC.isnull()])

# Duplicated MCC for TADIG

print('\nDuplicated MCC for TADIG:')
duplicated_mcc_list = [x[0] for x in
                       df_mcc_map.groupby('TADIG',as_index=False)['SPONSOR'].
                       count().values if x[1] > 1]
display(df_mcc_map[df_mcc_map.TADIG.isin(duplicated_mcc_list)])

# Prepare DataFrame with SPONSOR and MCC only

df_mcc = df_mcc_map[['SPONSOR','MCC']].drop_duplicates()

df_mcc['COUNTRY_NAME'] = '@'
df_mcc['Profile'] = profile

#*******************************************************************
# Adjustments if needed

# df_mcc.loc[df_mcc.MCC.isin(['455','452','520','219','297','294']),'SPONSOR'] = 'MB'

#*******************************************************************

# Check for Duplicated MCC

print('\nDuplicated MCC:')
duplicated_mcc_list = [x[0] for x in
                       df_mcc.groupby(['SPONSOR','MCC'], as_index=False)['Profile'].
                       count().values if x[2] > 1]
display(df_mcc[df_mcc.MCC.isin(duplicated_mcc_list)])

# The result

print('\nMCC MAP list (first raws):')
display(df_mcc.head(3))
display(df_mcc.groupby('SPONSOR')['MCC'].count())

Missing MCC:


,SPONSOR,TADIG,MCC



Duplicated MCC for TADIG:


,SPONSOR,TADIG,MCC



Duplicated MCC:


,SPONSOR,MCC,COUNTRY_NAME,Profile



MCC MAP list (first raws):


,SPONSOR,MCC,COUNTRY_NAME,Profile
0,MB_STI,602,@,STI_COMBINE_MAP
1,Partner_STI,460,@,STI_COMBINE_MAP
2,Partner_STI,450,@,STI_COMBINE_MAP


SPONSOR
MB_STI          5
P4_STI          4
Partner_STI    89
S6_STI_STS     35
Name: MCC, dtype: int64

In [ ]:
# Create TOC file to CREATE MCC MAP list

result_file_name = f'{profile.lower()}_toc_create.rcjson'

ouf=open(join(downloads, result_file_name),'w')
#ouf.write('Some text\n')
#ouf.write(str(25))

num_val = df_mcc.count()[0]-1

t1='dmi_mcc_map.mcc_map.create,COUNTRY="",MCC="'
t2='",PROFILE_REF="'
t3='",SPONSOR_REF="'
t4='";\n'

for i, [sponsor, country, mcc, profile] in enumerate(df_mcc[['SPONSOR','COUNTRY_NAME','MCC','Profile']].values):
    ouf.write(t1+str(mcc)+t2+profile+t3+sponsor+t4)

ouf.close()

In [40]:
# Create Batch file to DELETE MCC MAP list

result_file_name = 'mcc_map_delete.rcjson'

file_mcc_map_dmi = 'DMI MCC MAP.dmi_mcc_map_mcc_map_export_Fri_Oct_30_2020.csv'

df_mcc_delete = pd.read_csv(join(downloads, file_mcc_map_dmi))
df_mcc_delete = df_mcc_delete.loc[df_mcc_delete.Profile == profile]

df_mcc_delete.rename({'Sponsor reference':'SPONSOR','Country':'COUNTRY_NAME'}, axis=1, inplace=True)

df_mcc_delete.COUNTRY_NAME.fillna('', inplace=True)

ouf=open(join(downloads, result_file_name), 'w')

num_val = df_mcc_delete.count()[0]-1

t1='{"params":{"country":"'
t2='","mcc":"'
t3='","profile":"'
t4='","sponsorReference":"'
t5='"},"type":"remove","caption":"dmi_mcc_map_mcc_map","objectId":"DMI MCC MAP.dmi_mcc_map_mcc_map","serviceId":"DMI MCC MAP"}'

ouf.write('[')
for i, [sponsor, country, mcc, profile] in enumerate(df_mcc_delete[['SPONSOR','COUNTRY_NAME','MCC','Profile']].values):
    ouf.write(t1+str(country)+t2+str(mcc)+t3+profile+t4+sponsor+t5)
    if i < num_val:
        ouf.write(',')
ouf.write(']')

ouf.close()

df_mcc_delete.groupby(['Profile','SPONSOR'], as_index=False)['MCC'].count()

,Profile,SPONSOR,MCC
0,STI_COMBINE_MAP,MB_STI,5
1,STI_COMBINE_MAP,P4_STI,25
2,STI_COMBINE_MAP,Partner_STI,81
3,STI_COMBINE_MAP,S6_STI_STS,3


### Create SX_ALL allowed list

In [68]:
#-----------------------------------------
# Set variables here

sponsor = 'S8'

al_all_file = 's8_al_missing.csv'
# The example of the content of the file with the list of allowed networks:
# TADIG,SPONSOR,NETWORK_ID
# ALBVF,S1,111
# ARGCM,S5,222
# ARM01,S2,333

output_file_csv_with_missing = 'df_al_create.csv'
result_file_name_al_create = 'al_create.rcjson' # file name with TOC comands for cerating the new Allowed list

#-----------------------------------------

# Download data from file

df_missing = pd.read_csv(join(downloads, al_all_file))

# Select only discounted

disc_list = df_disc.loc[df_disc.SPONSOR == sponsor, 'NETWORK_ID'].to_numpy(dtype='int64')
df_missing = df_missing.loc[df_missing.NETWORK_ID.isin(disc_list)]

# Select TADIGs from all ALL allowed lists of all sponsors

df_tadig = DataFrame()
for df in df_al_s_list:
    df_tadig = df_tadig.append(df[['TADIG','NETWORK_ID']])
    df_tadig.drop_duplicates(inplace=True)

# Prepare the DataFrame for creating the allowed list

df_al_new_create = pd.merge(df_missing, df_tadig,
                            how='left', on='NETWORK_ID', suffixes=['', '_AL'])[['SPONSOR','TADIG_AL']]

df_al_new_create['MOC'] = 1
df_al_new_create['3G_ONLY'] = 0
df_al_new_create.dropna(inplace=True)
# df_al_new_create.rename({'TADIG_AL':'TADIG'}, axis=1, inplace=True)
df_al_new_create.to_csv(join(downloads, output_file_csv_with_missing), index=False)


# Prepare TOC commands to add missing TADIGs
prepare_toc_to_create_al(df_al_new_create, downloads, result_file_name_al_create)

print('Missing:')
display(df_al_new_create)

Missing:


,SPONSOR,TADIG_AL,TADIG_AL,MOC,3G_ONLY


### Developments